In [4]:
"""
Utility module for converting Python functions to API-compatible formats.
"""

import inspect
import functools
from typing import Any, Dict, List, Optional, Callable, Union, get_type_hints, get_origin, get_args


class FunctionUtils:
    """
    Utilities for working with functions and API representations.
    """
    
    @staticmethod
    def python_to_gemini_function(
        function: Callable,
        name: Optional[str] = None,
        description: Optional[str] = None
    ) -> Dict[str, Any]:
        """
        Convert a Python function to a Gemini API function declaration.
        
        Args:
            function: Python function to convert
            name: Optional custom name for the function
            description: Optional custom description
            
        Returns:
            Function declaration in Gemini API format
        """
        # Get function name and description
        func_name = name or function.__name__
        func_description = description or inspect.getdoc(function) or ""
        
        # Get function signature and type hints
        sig = inspect.signature(function)
        type_hints = get_type_hints(function)
        
        # Build parameters schema
        parameters = {
            "type": "OBJECT",
            "properties": {},
            "required": []
        }
        
        # Process each parameter
        for param_name, param in sig.parameters.items():
            # Skip self parameter for class methods
            if param_name == "self":
                continue
                
            # Get parameter type and convert to Gemini schema type
            param_type = type_hints.get(param_name, str)  # Default to string
            schema_type = FunctionUtils._python_type_to_gemini_type(param_type)
            
            # Add parameter to properties
            parameters["properties"][param_name] = schema_type
            
            # If no default value, mark as required
            if param.default == inspect.Parameter.empty:
                parameters["required"].append(param_name)
        
        # Build function declaration
        function_declaration = {
            "name": func_name,
            "description": func_description,
            "parameters": parameters
        }
        
        # Add return type if available
        if "return" in type_hints:
            return_type = type_hints["return"]
            if return_type is not None:
                function_declaration["response"] = FunctionUtils._python_type_to_gemini_type(return_type)
        
        return function_declaration
    
    @staticmethod
    def _python_type_to_gemini_type(python_type: Any) -> Dict[str, Any]:
        """
        Convert a Python type to a Gemini API schema type.
        
        Args:
            python_type: Python type annotation
            
        Returns:
            Schema type in Gemini API format
        """
        # Handle primitive types
        if python_type == str:
            return {"type": "STRING"}
        elif python_type == int:
            return {"type": "INTEGER"}
        elif python_type == float:
            return {"type": "NUMBER"}
        elif python_type == bool:
            return {"type": "BOOLEAN"}
        
        # Handle Union types (e.g., Optional)
        origin = get_origin(python_type)
        if origin is Union:
            args = get_args(python_type)
            # Handle Optional (Union with None)
            if type(None) in args:
                non_none_args = [arg for arg in args if arg is not type(None)]
                if len(non_none_args) == 1:
                    base_type = FunctionUtils._python_type_to_gemini_type(non_none_args[0])
                    base_type["nullable"] = True
                    return base_type
            
            # Handle general Union as anyOf
            schemas = [FunctionUtils._python_type_to_gemini_type(arg) for arg in args 
                      if arg is not type(None)]
            return {"anyOf": schemas}
        
        # Handle List and other container types
        if origin is list or python_type == List:
            args = get_args(python_type)
            item_type = str  # Default item type
            if args:
                item_type = args[0]
            return {
                "type": "ARRAY",
                "items": FunctionUtils._python_type_to_gemini_type(item_type)
            }
        
        if origin is dict or python_type == Dict:
            return {"type": "OBJECT"}
        
        # Default to string for unknown types
        return {"type": "STRING"}

    @staticmethod
    def prepare_functions_for_api(functions: List[Callable]) -> List[Dict[str, Any]]:
        """
        Convert a list of Python functions to API-compatible format.
        
        Args:
            functions: List of functions to convert
            
        Returns:
            List of function declarations in API format
        """
        return [FunctionUtils.python_to_gemini_function(func) for func in functions]